### Example snippet to move the IMDA National Speech Corpus data from Dropbox to Azure Blob Storage

IMDA Speech corpus data is found in dropbox here: 

https://www.imda.gov.sg/NationalSpeechCorpus

create similar directories called part1>scripts part1>wave part2>scripts and part 2>wave

In [ ]:
import pandas as pd
import numpy as np
import os,sys,json
import matplotlib.pyplot as plt
import glob
from IPython import InteractiveShell

InteractiveShell.ast_node_interactivity="all"

#### MOVE FILES FROM DROPBOX TO AZURE BLOB STORAGE

In [ ]:
import dropbox

dbx = dropbox.Dropbox("ACCESS TOKEN")

import zipfile


In [ ]:
part=['part1','part2']
channel=[0,1,2]
option=['script','wave']

path='/imda - national speech corpus/'+part[0]+'/data'

s_paths=[] ##Get all possible script paths
w_paths=[] ##Get all possible wave paths
for p in part:
    for c in channel:
        script_path='/imda - national speech corpus/'+p+'/data/channel'+str(c)+'/script'
        wave_path='/imda - national speech corpus/'+p+'/data/channel'+str(c)+'/wave'
        s_paths.append(script_path)
        w_paths.append(wave_path)




### Exract all transcripts and send it to blob storage



In [ ]:
AZURE_ACC_NAME = 'ENTER AZURE ACCOUNT NAME'
AZURE_PRIMARY_KEY = 'AZURE STORAGE ACCOUNT KEY'
import zipfile
import glob
import shutil

from azure.storage.blob.baseblobservice import BaseBlobService
from azure.storage.blob import (
    BlockBlobService,
    ContainerPermissions,
    BlobPermissions,
    PublicAccess,
)

block_blob_service = BlockBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)

##Write all transcripts from PART 1

for s in s_paths[:3]:
    ent=dbx.files_list_folder(s).entries
    for e in ent:
        file_name='part1/script/'+e.name
        with open(file_name,'wb') as file:
            met,res=dbx.files_download(e.path_lower)
            file.write(res.content)
            block_blob_service.create_blob_from_path(container_name='audio',blob_name=file_name, file_path=file_name)

shutil.rmtree(os.path.join(os.getcwd(),'part1','script'))

##Write all transcripts from PART 2
for s in s_paths[3:]:
    ent=dbx.files_list_folder(s).entries
    for e in ent:
        file_name='part2/script/'+e.name
        with open(file_name,'wb') as file:
            met,res=dbx.files_download(e.path_lower)
            file.write(res.content)
            block_blob_service.create_blob_from_path(container_name='audio',blob_name=file_name, file_path=file_name)

shutil.rmtree(os.path.join(os.getcwd(),'part2','script'))

In [ ]:
##WRITE ZIP FILES FOR FIRST PART AUDIO

for w in w_paths[:3]:
    ent=dbx.files_list_folder(w).entries
    for e in ent:
        file_name='part1/wave/'+e.name       
        dbx.files_download_to_file(file_name,e.path_lower)
        zip_ref = zipfile.ZipFile(file_name, 'r')
        zip_ref.extractall('part1/wave/')
        zip_ref.close()
        sound_path='part1/wave/'+e.name[:-4]
        for file in glob.glob(os.path.join(sound_path,'SESSION0','*.wav')):
            block_blob_service.create_blob_from_path(container_name='audio',blob_name=file, file_path=file)
        for file in glob.glob(os.path.join(sound_path,'SESSION1','*.wav')):
            block_blob_service.create_blob_from_path(container_name='audio',blob_name=file, file_path=file)
            os.remove(os.path.join(os.getcwd(),'part1','wave',e.name[:-4])+'.zip')
            shutil.rmtree(os.path.join(os.getcwd(),'part1','wave',e.name[:-4]))
            
    

In [ ]:
##WRITE ZIP FILES FOR SECOND PART AUDIO

for w in w_paths[3:]:
    ent=dbx.files_list_folder(w).entries
    for e in ent:
        file_name='part2/wave/'+e.name       
        dbx.files_download_to_file(file_name,e.path_lower)
        zip_ref = zipfile.ZipFile(file_name, 'r')
        zip_ref.extractall('part2/wave/')
        zip_ref.close()
        sound_path='part2/wave/'+e.name[:-4]
        for file in glob.glob(os.path.join(sound_path,'SESSION0','*.wav')):
            block_blob_service.create_blob_from_path(container_name='audio',blob_name=file, file_path=file)
        for file in glob.glob(os.path.join(sound_path,'SESSION1','*.wav')):
            block_blob_service.create_blob_from_path(container_name='audio',blob_name=file, file_path=file)
            os.remove(os.path.join(os.getcwd(),'part2','wave',e.name[:-4])+'.zip')
            shutil.rmtree(os.path.join(os.getcwd(),'part2','wave',e.name[:-4]))
            
    